In [1]:
'''
basing the dev of this off of: https://www.llama.com/docs/model-cards-and-prompt-formats/meta-llama-guard-2/

stealing this code: https://huggingface.co/meta-llama/Meta-Llama-Guard-2-8B
'''

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-Guard-2-8B"
device = "cuda"
dtype = torch.bfloat16

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, 
                                             torch_dtype=dtype, 
                                             load_in_4bit=True, #only need this for itty bitty devices like mine
                                             device_map=device)



# moderate([
#     {"role": "user", "content": "I forgot how to kill a process in Linux, can you help?"},
#     {"role": "assistant", "content": "Sure! To kill a process in Linux, you can use the kill command followed by the process ID (PID) of the process you want to terminate."},
# ])

/mnt/c/Users/hew7/documents/venvs/pythia/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 4/4 [00:16<00:00,  4.17s/it]


In [6]:
def moderate(input_str: str):
    chat = [{"role": "user", "content": input_str},]
    input_ids = tokenizer.apply_chat_template(chat, return_tensors="pt").to(device)
    output = model.generate(input_ids=input_ids, max_new_tokens=100, pad_token_id=0)
    prompt_len = input_ids.shape[-1]
    return tokenizer.decode(output[0][prompt_len:], skip_special_tokens=True)

In [11]:
moderate('women are terrible/')

'unsafe\nS9'

In [13]:
print(tokenizer.chat_template)

{% if messages|length % 2 == 0 %}{% set role = 'Agent' %}{% else %}{% set role = 'User' %}{% endif %}{{ "[INST] Task: Check if there is unsafe content in '" + role + "' messages in conversations according our safety policy with the below categories.

<BEGIN UNSAFE CONTENT CATEGORIES>
S1: Violent Crimes. 
S2: Non-Violent Crimes. 
S3: Sex Crimes. 
S4: Child Exploitation. 
S5: Specialized Advice. 
S6: Privacy. 
S7: Intellectual Property. 
S8: Indiscriminate Weapons. 
S9: Hate. 
S10: Self-Harm. 
S11: Sexual Content. 
<END UNSAFE CONTENT CATEGORIES>

<BEGIN CONVERSATION>

" }}{% for message in messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{% set role = 'User' %}{% elif message['role'] == 'assistant' %}{% set role = 'Agent' %}{% endif %}{{ role + ': ' + content.strip() + '

' }}{% endfor %}{{ 